In [1]:
from PIL import Image
import pytesseract
from pytesseract import Output
import numpy as np
import re
import cv2
import spacy

#cv2.imshow('image',image) # dipalying the image
#cv2.waitKey(0) 
#cv2.destroyAllWindows()

#text = pytesseract.image_to_string(image)
#print(text)

def ocr_core(img):
    text = pytesseract.image_to_string(img)
    return text

img= cv2.imread('invoice.jpg')  #reading the image
    
#PREPROCESSING THE IMAGE

def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#image1=get_grayscale(image)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,3)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 100, 230, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation = to adjust the font sizes
def dilate(image):
    kernel = np.ones((2,2),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion = = to adjust the font sizes
def erode(image):
    kernel = np.ones((2,2),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((1,1),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated


#canny edge detection == NOT RECOMMENDED RESULTING IN ACCURACY DROP
#def canny(image):
#    return cv2.Canny(image, 100, 200)

img= get_grayscale(img)
img= thresholding(img)
img= remove_noise(img)
img=erode(img)
img=dilate(img)
img= opening(img)


d = pytesseract.image_to_data(img, output_type=Output.DICT)
#print(d.keys())

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 2)


        


#cv2.imshow('img', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

#d = pytesseract.image_to_data(img, output_type=Output.DICT)
#n_boxes = len(d['level'])
#for i in range(n_boxes):

#    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])    
#    img = cv2.rectangle(img, (x, y), (x + w, y + h), (255,0,0), 2)




#custom_oem_psm_config =r'--oem 3 --psm 6'
#print(ocr_core(img))





text = pytesseract.image_to_string(img)
nlp = spacy.load("en_core_web_sm")
sents = nlp(text)
count =1
data_list = []
# Text Labelling on entities
for sent in sents:
    #print(sent)
    tokens = sent.text.split(" ")
    for i in range(len(tokens)):
        var = tokens[i]
        data = {}
        # print(var)
        if re.match(r'^[0-9]{2}[A-Z]{5}[0-9]{4}[A-Z]{1}[1-9A-Z]{1}Z[0-9A-Z]{1}$', str(var)):
            print("GST Number", str(var))
            data.update({"GST Number": str(var)})
        elif re.search(r'^[a-zA-Z]{5}[0-9]{4}[a-zA-Z]$', re.sub(r'\s+', '', str(var))):
                print("PAN Number", str(var))
                data.update({"PAN Number": str(var)})
            # Using Regular expression to match dates
        elif re.match(r'^(0[1-9]|[12][0-9]|3[01]).(0[1-9]|1[012]).(19|20)\d\d$', str(var)):
            print("Date of Invoice", str(var))
            data.update({"Date of Invoice": str(var)})
            #Simple number for amount
        elif re.match(r'\d{1,6}', str(var)):
            print("Amount", str(var))
            data.update({"Amount" : str(var)})

        count = count + 1
        if data != {}:
            data_list.append(data)

print(data_list)



Amount 938
Amount 23651891
Amount 29
Amount 01
Amount 2029
Amount 1
Amount 1
Amount 33
Amount 1
GST Number 33AAACT5557G1ZE
Amount 54
Amount 24243.000
Amount 6S
Amount 20
Amount 32ET00N
Amount 147,107.09
Amount 108.00
Amount 7
Amount 35,999.44
Amount 11,108.00
Amount 235990
Amount 44
Amount 50
Amount 0
Amount 29.411
Amount 2020
Amount 38807869701
Amount 56807869703
Amount 1
Amount 2
Amount 5
Amount 59
[{'Amount': '938'}, {'Amount': '23651891'}, {'Amount': '29'}, {'Amount': '01'}, {'Amount': '2029'}, {'Amount': '1'}, {'Amount': '1'}, {'Amount': '33'}, {'Amount': '1'}, {'GST Number': '33AAACT5557G1ZE'}, {'Amount': '54'}, {'Amount': '24243.000'}, {'Amount': '6S'}, {'Amount': '20'}, {'Amount': '32ET00N'}, {'Amount': '147,107.09'}, {'Amount': '108.00'}, {'Amount': '7'}, {'Amount': '35,999.44'}, {'Amount': '11,108.00'}, {'Amount': '235990'}, {'Amount': '44'}, {'Amount': '50'}, {'Amount': '0'}, {'Amount': '29.411'}, {'Amount': '2020'}, {'Amount': '38807869701'}, {'Amount': '56807869703'}, {'Am

In [1]:

#Fixing orientation
##
#Applying median blur
#image_blur=cv2.GaussianBlur(image2,(3,3),cv2.BORDER_DEFAULT)
#increaseing the size of our image
#image1=cv2.resize(image_blur, None, fx=2.0,fy=2.0,interpolation=cv2.INTER_CUBIC)


In [ ]:
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x5613c01683a0) is not the object's thread (0x5613c10a8bf0).
Cannot move to target thread (0x5613c01683a0)

QObject::moveToThread: Current thread (0x5613c01683a0) is not the object's thread (0x5613c10a8bf0).
Cannot move to target thread (0x5613c01683a0)

QObject::moveToThread: Current thread (0x5613c01683a0) is not the object's thread (0x5613c10a8bf0).
Cannot move to target thread (0x5613c01683a0)

QObject::moveToThread: Current thread (0x5613c01683a0) is not the object's thread (0x5613c10a8bf0).
Cannot move to target thread (0x5613c01683a0)

QObject::moveToThread: Current thread (0x5613c01683a0) is not the object's thread (0x5613c10a8bf0).
Cannot move to target thread (0x5613c01683a0)

QObject::moveToThread: Current thread (0x5613c01683a0) is not the object's thread (0x5613c10a8bf0).
Cannot move to target thread (0x5613c01683a0)

QObject::moveToThread: Current thread (0x5613c01683a0) is not the object's thread (0x5613c10a8bf0).
Cannot move to tar